In [1]:
import scipy.stats as scs
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import date, datetime, timedelta
from collections import Counter
plt.style.use('fivethirtyeight')
sns.set_style('darkgrid')
%matplotlib inline

In [3]:
import os

In [4]:
os.listdir('data')

['data_with_usd.csv',
 'data_nobadrows.csv',
 'all_data.csv',
 'data_with_pos',
 'ks-projects-201801.csv',
 'ks-projects-201612.csv',
 'exchange.csv']

In [41]:
df = pd.read_csv('data/data_with_dummies_and_class.csv')

In [42]:
keep_cols = [x for x in df.columns if 'Unnamed' not in x]
df = df[keep_cols].set_index('ID')

In [43]:
df.shape

(693919, 249)

In [44]:
df.sample(5)

,goal,pledged,backers,usd pledged,name$word_count,name$punc_count,name$caps_count,name$possesive_count,name$noun_count,name$adj_count,...,launched_month&Jul,launched_month&Feb,launched_month&Dec,launched_month&Apr,launched_month&Sep,launched_month&Oct,launched_month&May,launched_month&Nov,launched_month&Jun,success
ID,,,,,,,,,,,,,,,,,,,,,
253783563,1000,1575,23,1575.00,3,0.000000,0.187500,0.0,0.666667,0.000000,...,False,False,False,False,False,False,False,False,False,True
781254659,1350000,81,4,81.00,4,0.000000,0.190476,0.0,0.250000,0.250000,...,True,False,False,False,False,False,False,False,False,False
292318024,10000,712,14,712.00,4,0.037037,0.074074,0.0,0.500000,0.250000,...,False,False,False,True,False,False,False,False,False,False
1683692425,25000,1,1,1.00,2,0.000000,0.181818,0.0,1.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
1066891227,5000,34026,1267,34026.89,6,0.000000,0.160000,0.0,0.500000,0.166667,...,True,False,False,False,False,False,False,False,False,True


In [45]:
features = Counter()
for col in df.columns:
    features[col.split('&')[0].strip()] += 1
features = pd.Series(features)

In [64]:
features

backers                      1
category                   159
country                     22
currency                    14
deadline_month              12
goal                         1
launched_month              12
length                       1
main_category               15
name$adj_count               1
name$caps_count              1
name$determinator_count      1
name$noun_count              1
name$possesive_count         1
name$preposition_count       1
name$punc_count              1
name$verb_count              1
name$word_count              1
pledged                      1
success                      1
usd pledged                  1
dtype: int64

In [49]:
dfl = df.drop(['usd pledged', 'pledged'], axis=1)

In [53]:
df_logis = dfl.drop([col for col in dfl.columns if col.split('&')[0] == 'category'], axis=1)

In [79]:
df_logis = df_logis.dropna()

In [56]:
from sklearn import linear_model as lm

In [57]:
model = lm.LogisticRegression(C=.8)

In [63]:
cv = pd.Series(np.random.choice(range(5), df_logis.shape[0]), index=df_logis.index)

In [85]:
df_logis.index.shape

(693917,)

In [84]:
df_logis.index.unique().shape

(374863,)

In [81]:
Xtr = df_logis.loc[cv != 0, :]
Xte = df_logis.loc[cv == 0, :]
ytr = Xtr.pop('success')
yte = Xte.pop('success')

ValueError: cannot reindex from a duplicate axis

In [ ]:
model.fit(Xtr, Xte)

In [75]:
Xtr[np.isnan(Xtr['name$punc_count'])]

,goal,backers,name$word_count,name$punc_count,name$caps_count,name$possesive_count,name$noun_count,name$adj_count,name$verb_count,name$preposition_count,...,launched_month&Mar,launched_month&Jul,launched_month&Feb,launched_month&Dec,launched_month&Apr,launched_month&Sep,launched_month&Oct,launched_month&May,launched_month&Nov,launched_month&Jun
ID,,,,,,,,,,,,,,,,,,,,,


In [78]:
Xtr[Xtr['name$punc_count'] == np.inf]

,goal,backers,name$word_count,name$punc_count,name$caps_count,name$possesive_count,name$noun_count,name$adj_count,name$verb_count,name$preposition_count,...,launched_month&Mar,launched_month&Jul,launched_month&Feb,launched_month&Dec,launched_month&Apr,launched_month&Sep,launched_month&Oct,launched_month&May,launched_month&Nov,launched_month&Jun
ID,,,,,,,,,,,,,,,,,,,,,
518903190,3500,84,0,inf,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,False,False,False,False,False,False,False,False
